In [3]:
import logging
import gensim
# Login
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# loading model
model = gensim.models.KeyedVectors.load_word2vec_format('./model/GoogleNews-vectors-negative300.bin', binary=True)

2017-12-02 17:32:56,277 : INFO : loading projection weights from ./model/GoogleNews-vectors-negative300.bin
2017-12-02 17:33:56,288 : INFO : loaded (3000000, 300) matrix from ./model/GoogleNews-vectors-negative300.bin


In [4]:
print(model.most_similar(positive=['forest', 'brick'], negative=['tree'], topn=3))

2017-12-02 17:33:56,335 : INFO : precomputing L2-norms of word weight vectors


[('stucco', 0.41667330265045166), ('mortar_booksellers', 0.41204798221588135), ('tile_flooring', 0.4049796462059021)]


In [5]:
more_examples = ["he his she", "big bigger bad", "going went being"]
for example in more_examples:
    a, b, x = example.split()
    predicted = model.most_similar([x, b], [a])[0][0]
    print(predicted)

her
worse
were


In [6]:
more_examples = ["he his she", "big bigger bad", "going went being"]
for example in more_examples:
    a, b, x = example.split()
    predicted = model.most_similar([x, b], [a])[0][0]
    # print(predicted)
    print ('({} is to {}) as ({} is to {})'.format(a, b, x, predicted))

(he is to his) as (she is to her)
(big is to bigger) as (bad is to worse)
(going is to went) as (being is to were)


In [34]:
word_list = ["sing song", "drink water", "read book", "eat food", "wear coat", "drive car", "ride horse", "give gift", "attack enemy", "say word", "open door", "climb tree", "heal wound", "cure disease", "paint picture"]
# canons should be a list of pair of "noun verb"
def more_verb (conons, n2):
    m = 0
    sigma = 0
    for pair in conons:
        v1, n1 = pair.split()
        sigma += model.word_vec(v1) - model.word_vec(n1)
        m += 1
        predicted = model.most_similar([n2, v1], [n1])[0][0]
        print ('{} : {} :: [{}] : {}'.format(v1, n1, predicted, n2))
    a = (1/m)*sigma
    predicted = model.most_similar([a, n2], [])
    print("--------------------------------------------")
    for i in range(10):
        print ('canon :: [{}] : {})'.format(predicted[i][0], n2))
more_verb(word_list, "horse")

sing : song :: [horses] : horse
drink : water :: [horses] : horse
read : book :: [horses] : horse
eat : food :: [horses] : horse
wear : coat :: [horses] : horse
drive : car :: [horses] : horse
ride : horse :: [rides] : horse
give : gift :: [horses] : horse
attack : enemy :: [horses] : horse
say : word :: [horses] : horse
open : door :: [horses] : horse
climb : tree :: [horses] : horse
heal : wound :: [horses] : horse
cure : disease :: [racehorse] : horse
paint : picture :: [horses] : horse
--------------------------------------------
canon :: [gallop] : horse)
canon :: [ride] : horse)
canon :: [horses] : horse)
canon :: [overrace] : horse)
canon :: [take] : horse)
canon :: [viruses_spyware_Trojan] : horse)
canon :: [outkick] : horse)
canon :: [Trainer_Jinks_Fires] : horse)
canon :: [go] : horse)
canon :: [Koriner] : horse)


In [16]:
def predict_words (conons, word, word_position):
    m = 0
    sigma = 0
    for pair in conons:
        w1, w2 = pair.split()
        sigma += model.word_vec(w1) - model.word_vec(w2)
        m += 1
    a = (1/m)*sigma
    if word_position == 0:
        predicted = model.most_similar([word], [a], topn = 10)
        return predicted
    elif word_position == 1:
        predicted = model.most_similar([a, word], [], topn = 10)
        return predicted
    else:
        print("please put in correct cannon position")
        
predict_words(n_adj, "pen", 0)


# run for a test 
def main():
    n_adj = ["knife sharp", "light bright", "ice cold", "fire burning", "desert dry", "sky blue", "night dark", "rope long"]
    verb_n = ["sing song", "drink water", "read book", "eat food", "wear coat", "drive car", "ride horse", "give gift",
                 "attack enemy", "say word", "open door", "climb tree", "heal wound", "cure disease", "paint picture"]
#     demo_more_verb(word_list, "book")
    list = ["book", "day", "love"]
    for word in list:
        print("")
        print("predict verb on", word)
        print(predict_words(verb_n, word , 1))
        print("predict adj on", word)
        print(predict_words(n_adj, word , 0))
if __name__ == "__main__": main()


predict verb on book
[('write', 0.486385703086853), ('read', 0.46074622869491577), ('take', 0.41436272859573364), ('publish', 0.4090305268764496), ('eat', 0.40674889087677), ('Gordon_Korman', 0.4022621214389801), ('devour', 0.39084985852241516), ('books', 0.38471636176109314), ('teach', 0.3764316439628601), ('tome', 0.37393248081207275)]
predict adj on book
[('books', 0.46271762251853943), ('bestseller', 0.4351135194301605), ('memoir', 0.43337810039520264), ('Great_Deluge', 0.43230074644088745), ('memoirs', 0.4283945560455322), ('eminently_readable', 0.4201478362083435), ('Reading_Lolita', 0.4165016710758209), ('paperback', 0.4161180853843689), ('Jenny_Diski', 0.412182092666626), ('hardbound_edition', 0.40317094326019287)]

predict verb on day
[('take', 0.42790526151657104), ('go', 0.4151778221130371), ('week', 0.4002237617969513), ('eat', 0.3987203538417816), ('wear', 0.397201806306839), ('lift_weights', 0.3869890868663788), ('ride_stationary_bike', 0.38589006662368774), ('morning_Th

In [36]:
import spacy
import time
def pre_possible_action(sentence):
    nlp = spacy.load('en')
    doc = nlp(sentence)
    canons = ["sing song", "drink water", "read book", "eat food", "wear coat", "drive car", "ride horse", "give gift", "attack enemy", "say word", "open door", "climb tree", "heal wound", "cure disease", "paint picture"]
    dictionary = {}
    for chunk in doc.noun_chunks:
        word = chunk.root.text
        if word not in dictionary:
            dictionary[word] = predict_words(canons, word, 1)
    return dictionary

def pretty_print_dict(dictionary):
    print("")
    for key, val in dictionary.items():
        print (key, ":", val)

def main():
    tic = time.time()
    s = "Soon you’ll be able to send and receive money from friends and family right in Messages."
    s1 = "This is an open field west of a white house, with a boarded front door. There is a small mailbox here."
    s2 = "This is a forest, with trees in all directions around you."
    s3 = "This is a dimly lit forest, with large trees all around.  One particularly large tree with some low branches stands here."
    sentences = [s, s1, s2, s3]
    for sentence in sentences:
        print("-" * 3, sentence, "-" * 3)
        pretty_print_dict(pre_possible_action_1(sentence))
    toc = time.time()
    print("total time spend:", toc - tic, "s")
    
if __name__ == "__main__": main()

--- Soon you’ll be able to send and receive money from friends and family right in Messages. ---
friends : [('socialize', 0.48138174414634705), ('buddies', 0.4502038061618805), ('enjoy', 0.4474574327468872), ('girlfriends', 0.43792399764060974), ('hang', 0.437337726354599), ('befriend', 0.4312940239906311), ('eat', 0.4285794794559479), ('sing_karaoke', 0.42660951614379883), ('relax', 0.42622271180152893), ('speak_unpleasant_truths', 0.42595842480659485)]
family : [('grieve', 0.42577826976776123), ('eat', 0.41429322957992554), ('remain_sexually_abstinent', 0.4003693163394928), ('friends', 0.3989845812320709), ('stay', 0.3970945477485657), ('recuperate', 0.3956299424171448), ('relax', 0.39187291264533997), ('enjoy', 0.39164507389068604), ('families', 0.3901360034942627), ('socialize', 0.3831459581851959)]
Messages : [('Telephone_messages', 0.4744691252708435), ('Costoyas', 0.3635384738445282), ('respond', 0.3627692759037018), ('speak', 0.3591277599334717), ('Joseph_Farzam', 0.35341167449